# Authors

- Ikram Kohil, 2019115
- Johnatan Gao, 2013298

In [1]:
from IPython import display
from base64 import b64decode

# Vector of metrics

To solve the problems posed in this laboratory, we will extract metrics from the AST. To do this, we will find in the Python class AST a method called "vectorize". This method returns a vector corresponding to the occurrences of each type of node: a concrete example of the applied transformation is available below.

In order to solve the problems in this lab, we need to extract the metrics of AST.